Below, we demonstrate distribution calibration of probabilistic outcomes over discrete output. 

At first, we import the necessary files. 

In [ ]:
import torch
import sys
sys.path.append('../..')

from torchuq.transform.distcal_discrete import *
from torchuq.evaluate.distribution_cal import *
from torchuq.dataset.classification import *
from sklearn.linear_model import LogisticRegression
from torchuq.evaluate import categorical
from matplotlib import pyplot as plt

For this demo, we use the [UCI Digit classification dataset](https://archive.ics.uci.edu/dataset/80/optical+recognition+of+handwritten+digits/).

In [2]:


uci_datasets = ['digits', 'adult', 'iris', 'breast-cancer']
subset_uci = ['digits']


Below, we use Logistic Regression as a simple base model to predict the probabilistic outcome $(p_0, p_1,..,p_{9})$ over the discrete output ranging from 0 to 9. 

We use object of the *DiscreteDistCalibrator* class to train a recalibrator that takes the probabilistic outcome from the base model and outputs the recalibrated distribution represented as $(p'_0, p'_1,..,p'_{9})$ . 

We use an independent calibration dataset to train the DiscreteDistCalibrator. We evaluate the quality of probabilistic uncertainty with calibration score as defined [here](https://arxiv.org/pdf/2112.07184). 



In [4]:
for name in subset_uci:
	# 60% Train, 20% Calibration, 20% Test dataset
	dataset = get_classification_datasets(name, val_fraction=0.2, test_fraction=0.2, split_seed=0, normalize=True, verbose=True)
	
	train_dataset, cal_dataset, test_dataset = dataset
	X_train, y_train = train_dataset[:][0], train_dataset[:][1]
	X_cal, y_cal = cal_dataset[:][0], cal_dataset[:][1]
	X_test, y_test = test_dataset[:][0], test_dataset[:][1]
	
	# Simple logistic regression classifier trained
	reg = LogisticRegression(random_state=0).fit(X_train, y_train)
	print("=="*25)
	print(f"Classification accuracy on Train: {reg.score(X_train, y_train):.2}")
	print(f"Classification accuracy on Test: {reg.score(X_test, y_test):.2}")
	print("=="*25)


	# Predict probabilistic outcome on K classes, on the calibration and test datasets 
	pred_cal = torch.Tensor(reg.predict_proba(X_cal.numpy()))

	pred_test = torch.Tensor(reg.predict_proba(X_test.numpy()))

	

	# Use the DiscreteDistCalibrator class and train it on the calibration dataset

	calibrator = DiscreteDistCalibrator(verbose=True)
	calibrator.train(pred_cal, torch.Tensor(y_cal))

	output_cal = calibrator(pred_cal)
	output_test = calibrator(pred_test)

	# Evaluation
	print("=="*25)
	print(f"[Calibration Dataset] Log loss before calibration = {discrete_cal_loss(y_cal, pred_cal)}; After calibration={discrete_cal_loss(y_cal, output_cal)}")
	print(f"[Calibration Dataset] Calibration score before calibration = {discrete_cal_score(y_cal, pred_cal)}, After calibration = {discrete_cal_score(y_cal, output_cal)}")

	print(f"[Test Dataset] Log loss before calibration = {discrete_cal_loss(y_test, output_test)}; After calibration={discrete_cal_loss(y_test, output_test)}")
	print(f"[Test Dataset] Calibration score before calibration = {discrete_cal_score(y_test, output_test)}, After calibration = {discrete_cal_score(y_test, output_test)}")
	print("=="*25)

Loading dataset digits....
Splitting into train/val/test with 1079/359/359 samples
Done loading dataset digits
Classification accuracy on Train: 1.0
Classification accuracy on Test: 0.96
[Calibration Dataset] Log loss before calibration = 0.11842121565418677; After calibration=0.10612225788495672
[Calibration Dataset] Calibration score before calibration = 0.11454739304221799, After calibration = 0.0562264395008079
[Test Dataset] Log loss before calibration = 0.1602416572992105; After calibration=0.1602416572992105
[Test Dataset] Calibration score before calibration = 0.07278033250383055, After calibration = 0.07278033250383055
